In [1]:
import glob
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import plotly.express as px
import pandas as pd
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.express.colors import sample_colorscale
from datetime import datetime, timedelta
import matplotlib as mpl
import matplotlib.animation as animation
import simplekml
import yaml
from scipy import constants
from scipy.optimize import minimize
from tqdm.contrib.concurrent import process_map

from utils.antenna import *
from utils.sig_processor import *   
from utils.vhcl_processor import * 
from utils.config_parser import *
from post_process import *
import json
import sigmf

EXP_DATE = "2023-12-15"
EXP_DIR = glob.glob(f"../field_data/measurements/{EXP_DATE}*")

print(f"Processing date: {EXP_DATE}")

## Parse this information from config file
sample_rate = 56e6
OFDM_LEN = 450
ZC_LEN = 401
OFDM_GUARD = 300

CC1 = (35.773851, -78.677010)
LW1 = (35.72747884, -78.69591754) 
LW2 = (35.72821230,-78.70093459)
H_TOWER_LW1 = 12
H_TOWER_LW2 = 10
UTC_TIME_OFF = 3
UTC_OFF_REQ = True

%matplotlib inline


Processing date: 2023-12-15


In [2]:
processor = PostProcessor() 

FORCE = False
processed = processor.process_dates(EXP_DIR, FORCE)

Processing measurements: 100%|██████████| 10/10 [00:02<00:00,  4.60it/s, Status=Mesaurement ../field_data/measurements/2023-12-15_16_53 already processed. Loading from cache.]


In [3]:
processed["meas"][0]

,index,time,center_freq,dist,h_dist,v_dist,wav_type,detected,power,avgPower,...,pitch,yaw,roll,vel_x,vel_y,vel_z,heading,speed,cropped_sig,vehicle
0,0,41.0,3564,33.330948,31.017933,12.200,ZC,True,"[0.5198724901430134, 0.7455266697484171, 0.805...",0.072636,...,-0.016111,-0.070807,-0.065637,0.01,0.0,0.0,0,0.01,"[0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0.004...",0 time 2...
1,0,41.400002,3564,33.326145,31.013559,12.198,ZC,True,"[0.6463272721306423, 0.8243898656814529, 0.823...",0.147299,...,-0.01617,-0.070659,-0.065511,0.01,0.0,0.0,0,0.01,"[0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (-0.0...",0 time 2...
2,0,41.799999,3564,33.324318,31.013559,12.193,ZC,True,"[0.5723088031827624, 0.7483598740524406, 0.794...",0.075104,...,-0.016086,-0.070648,-0.065526,0.01,0.01,0.0,0,0.014142,"[0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (0.01...",0 time 2...
3,0,43.0,3564,33.327656,31.017933,12.191,ZC,True,"[0.4789116388186747, 0.5804369865979987, 0.677...",-0.054894,...,-0.015927,-0.070453,-0.065893,0.0,0.0,0.0,0,0.0,"[0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (-0.0...",0 time 2...
4,0,42.200001,3564,33.321026,31.013559,12.184,ZC,True,"[0.44581141732450213, 0.6551841331829024, 0.66...",-0.050358,...,-0.016051,-0.070532,-0.065726,0.01,0.0,0.0,0,0.01,"[0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (-0.0...",0 time 2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063,0,466.200012,3564,33.887798,31.606663,12.223,ZC,True,"[-1.5275372620263203, -0.5878720322906884, -0....",-1.423917,...,-0.019563,-0.062378,-0.059623,0.0,-0.01,0.06,0,0.060828,"[0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (-0.0...",0 time 2...
1064,0,467.0,3564,33.887798,31.606663,12.223,ZC,True,"[-1.4486763828332714, -0.3842224432868814, -0....",-1.206070,...,-0.019563,-0.062378,-0.059623,0.0,-0.01,0.06,0,0.060828,"[0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (-0.0...",0 time 2...
1065,0,466.600006,3564,33.887798,31.606663,12.223,ZC,True,"[-1.3941854721212568, -0.3938581920844124, -0....",-1.276494,...,-0.019563,-0.062378,-0.059623,0.0,-0.01,0.06,0,0.060828,"[0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (0.00...",0 time 2...
1066,0,467.799988,3564,33.887798,31.606663,12.223,ZC,True,"[-1.127916583952464, -0.12588812019663115, -0....",-0.959733,...,-0.019563,-0.062378,-0.059623,0.0,-0.01,0.06,0,0.060828,"[0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (-0.0...",0 time 2...


In [4]:
processed["meas"][0].keys()

Index(['index', 'time', 'center_freq', 'dist', 'h_dist', 'v_dist', 'wav_type',
       'detected', 'power', 'avgPower', 'snr', 'avgSnr', 'snr_2', 'avgSnr_2',
       'freq_offset', 'path_loss', 'avg_pl', 'est_dist', 'peaks', 'orig_peaks',
       'start_point', 'aod_theta', 'aod_phi', 'aoa_theta', 'aoa_phi', 'stage',
       'corr', 'save_corr', 'rsrp', 'shadowing', 'multipath', 'delay',
       'doppler', 'type', 'lat', 'lon', 'alt', 'pitch', 'yaw', 'roll', 'vel_x',
       'vel_y', 'vel_z', 'heading', 'speed', 'cropped_sig', 'vehicle'],
      dtype='object')

In [52]:
processed["config"][0].WAV_OPTS.SEQ_LEN

401

In [8]:
processed["resultDir"][0]

'../field_data/post-results/2023-12-15_15_41/'

In [10]:
n_items = len(processed["meas"])

for i in range(n_items):
    out_dir = processed["resultDir"][i]
    out_dir = out_dir.replace("post-results", "sigMF")
    
    # Create the output folder if it doesn't exist
    os.makedirs(out_dir, exist_ok=True)
    for j, meas in processed["meas"][i].iterrows():
        timest = meas["vehicle"].iloc[0,0].timestamp()
        metadata_file = "Channel_Sounder_" + str(timest) + ".sigmf-meta"
        data_file = "Channel_Sounder_" + str(timest) + ".sigmf-data"

        metadata_path = os.path.join(out_dir, metadata_file)
        data_path = os.path.join(out_dir, data_file)
        
        sigmf_metadata = {
            "global": {
                "core:version": sigmf.__version__,
                "core:datatype": "f32_le", 
                "core:author": "https://aerpaw.org/",
                "core:description": "Air-to-Ground Channel Sounding Measurements",
                "core:sample_rate": processed["config"][i].USRP_CONF.SAMPLE_RATE,
                "core:num_channels": 1,
                "core:tx_gain_ref": 19.97, # dBm
                "core:rx_gain_ref": -50.68,
                "core:waveform": processed["config"][i].WAVEFORM,
                "core:zc_root_index": processed["config"][i].WAV_OPTS.ROOT_IND,
                "core:zc_len": processed["config"][i].WAV_OPTS.SEQ_LEN
            },
            "captures": [],
            "annotations": []
        }
        
        capture_entry = {
            "core:frequency": processed["config"][i].USRP_CONF.CENTER_FREQ,
            "core:time": meas["time"],
            "core:rx_location": {
                "latitude": meas["lat"],
                "longitude": meas["lon"],
                "altitude": meas["alt"]
            },
            "core:heading": meas["heading"],
            "core:rotation": {
                "pitch": meas["pitch"],
                "yaw": meas["yaw"],
                "roll": meas["roll"]
            },
            "core:velocity": {
                "velocity_x": meas["vel_x"],
                "velocity_y": meas["vel_y"],
                "velocity_z": meas["vel_z"]
            },
            "core:timestamp": meas["vehicle"].iloc[0,0].timestamp(),
            "core:tx_location": {
                "latitude" : LW1[0],
                "longitude" : LW1[1],
                "altitude" : H_TOWER_LW1
            },
            "core:flight_stage": meas["stage"],
            "core:speed": meas["speed"],
            "core:dist": meas["dist"]
        }
        
        sigmf_metadata["captures"].append(capture_entry)           
        data = meas["cropped_sig"]
        data.tofile(data_path)
        
        with open(metadata_path, 'w') as metadata_file:
            json.dump(sigmf_metadata, metadata_file, indent=4)